In [3]:
import orjson
import os
import pandas as pd
from tqdm import tqdm
import orjson
import numpy as np
import pickle

In [5]:
tier = 'diamond'
output_dir = "/home/piddle/hdd/matches"
target_dir = os.path.join(output_dir, tier)

In [3]:
path = os.path.join(target_dir, os.listdir(target_dir)[6])
file = orjson.loads(open(path, 'rb').read())
file

[{'id': 'CNrwP4AK2XecSzCd_03leKIrPIV4ObTdhmClTSh9hCA=',
  'memo': None,
  'created_at': '2024-02-17T17:59:11+09:00',
  'game_map': 'SUMMONERS_RIFT',
  'queue_info': {'id': 420,
   'queue_translate': '개인/2인 랭크 게임',
   'game_type': 'SOLORANKED'},
  'version': '14.3.558.8314',
  'game_length_second': 1366,
  'is_remake': False,
  'is_opscore_active': True,
  'is_recorded': False,
  'average_tier_info': {'tier': 'EMERALD',
   'division': 1,
   'tier_image_url': 'https://opgg-static.akamaized.net/images/medals_new/emerald.png',
   'border_image_url': 'https://opgg-static.akamaized.net/images/border_new/emerald.png'},
  'participants': [{'summoner': {'id': 3121752,
     'summoner_id': 'QInmWiCxLgX0_-HFlgl5HjGzn3fdN5tnPWglKT_Lpib39A',
     'acct_id': '8MmvWmq2KzPTXQvVFdJKQJbgOe--5knxt0Jb2ToXL1Kr',
     'puuid': 'xAU374T20XVUpcKaeoeLGUKdXugQS_xVfhDLocsFieKhKC48OEcOASOX3h0md6_cEd00aZ4dGccX5g',
     'game_name': '니꼬내꼬니코',
     'tagline': 'KR1',
     'name': '니꼬내꼬니코',
     'internal_name': '니꼬내꼬니

In [45]:
all_column=['match_id', 'game_length_second', 'summoner_id', 'summoner_level', 'champion_id', 'team_key', 'position',
             'trinket_item', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'rune_0', 'rune_1', 'spell_0', 'spell_1',
             'champion_level', 'damage_self_mitigated', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets',
             'total_damage_taken', 'total_damage_dealt', 'total_damage_dealt_to_champions', 'time_ccing_others',
             'vision_score', 'vision_wards_bought_in_game', 'sight_wards_bought_in_game', 'ward_kill', 'ward_place',
             'turret_kill', 'kill', 'death', 'assist', 'neutral_minion_kill', 'gold_earned', 'total_heal', 'result']
    
cols1 = ['champion_id', 'team_key', 'position', 'trinket_item']

stat_cols = ['champion_level', 'damage_self_mitigated', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets',
            'total_damage_taken', 'total_damage_dealt', 'total_damage_dealt_to_champions', 'time_ccing_others',
            'time_ccing_others', 'vision_wards_bought_in_game', 'sight_wards_bought_in_game', 'ward_kill', 'ward_place',
            'turret_kill', 'kill', 'death', 'assist', 'neutral_minion_kill', 'gold_earned', 'total_heal']

match_col = ['match_id', 'team_key', 'summoner_id', 'result']

In [46]:
def parse_match(file_iter) -> pd.DataFrame:
    match_by_match_id_chunk = []
    match_by_summoner_id_chunk = []
    for n, file in enumerate(tqdm(file_iter)):
        with open(file.path, 'rb') as f:
            try:
                json_data = orjson.loads(f.read())
            except Exception as e:
                print('Error reading {}: {}'.format(file.path, e))
                continue

        summoner_id = file.name[:-5]

        if not 'participants' in json_data[0]:
            continue
        
        for match in json_data:
            for participant in match['participants']:
                
                if match['game_length_second'] == 0:
                    continue
                
                match_by_match_id = {}

                match_by_match_id['match_id'] = match['id']
                match_by_match_id['summoner_id'] = participant['summoner']['summoner_id']
                match_by_match_id['team_key'] = participant['team_key']
                match_by_match_id['result'] = participant['stats']['result']

                match_by_match_id_chunk.append(match_by_match_id)

                if participant['summoner']['summoner_id'] == summoner_id:
                    match_by_summoner_id = {}
                    match_by_summoner_id['match_id'] = match['id']
                    match_by_summoner_id['game_length_second'] = match['game_length_second']

                    match_by_summoner_id['summoner_id'] = participant['summoner']['summoner_id']
                    match_by_summoner_id['summoner_level'] = participant['summoner']['level']

                    for col in cols1:
                        match_by_summoner_id[col] = participant[col]

                    for i, item in enumerate(participant['items']):
                        match_by_summoner_id[f'item_{i}'] = item

                    match_by_summoner_id['rune_0'] = participant['rune']["primary_rune_id"]
                    match_by_summoner_id['rune_1'] = participant['rune']["secondary_page_id"]
                    match_by_summoner_id['spell_0'] = participant['spells'][0]
                    match_by_summoner_id['spell_1'] = participant['spells'][1]

                    stats = participant['stats']
                    for col in stat_cols:
                        match_by_summoner_id[col] = stats[col]

                    match_by_summoner_id['vision_score'] = stats['vision_score']
                    match_by_summoner_id['result'] = stats['result']

                    match_by_summoner_id_chunk.append(match_by_summoner_id)


        if n % 20000 == 0:
            match_df = pd.DataFrame(match_by_match_id_chunk, columns=match_col)
            summoner_df = pd.DataFrame(match_by_summoner_id_chunk, columns=all_column)
            
            match_by_match_id_chunk = []
            match_by_summoner_id_chunk = []
            yield match_df, summoner_df

    else:
        match_df = pd.DataFrame(match_by_match_id_chunk, columns=match_col)
        summoner_df = pd.DataFrame(match_by_summoner_id_chunk, columns=all_column)
        yield match_df, summoner_df

In [47]:
file_iter = os.scandir(target_dir)
pd.DataFrame(columns=match_col).to_csv(f'../data/{tier}_match_by_match.csv', mode='w', index=False)
pd.DataFrame(columns=all_column).to_csv(f'../data/{tier}_match_by_summoner.csv', mode='w', index=False)

for match_df, summoner_df in parse_match(file_iter):
    match_df.to_csv(f'../data/{tier}_match_by_match.csv', mode='a', index=False, header=False)
    summoner_df.to_csv(f'../data/{tier}_match_by_summoner.csv', mode='a', index=False, header=False)

82586it [25:58, 52.98it/s]


# SUMMONER

In [28]:
target = os.listdir(target_dir)
for i, t in enumerate(target):
    target[i] = t[:-5]

target

['Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4',
 'bhFsgFImdcOrpTFZfh1SqtcQwGgpVmi2-oZJieaPbP7H8kQF9t7oQMD7Sg',
 'JwnzSf5qMZGsMlJvnG1tvBi54_9gmxATP4gKQuqOPLg2cqg',
 'xZXo-E0xfegWrTTHlhZtqLqCEb1J8NIipmJHdyvXxWcmUJ1xTXaGmV80sw',
 '6TUa8lY_iQBCMb7U1fftji3KLGq3FFcYLRTTFP2A5OTSqQ',
 'xfuG4x8DLz3FpzeH-DsyZkZSiDmR6RLQCxJFAtrHrii3zDE',
 'mTJGwAgsaMpgpAYlUOLG1wJPrnRJ330kljxQCccFY-7PNxI',
 'ep26dacWRld_1njrzHD_d4CKd0-V9XKMDGdkA8EGIQr-BDuQw-RCcGaDcQ',
 'cbnOhGaHjnGqqravIqDqAcDbyQKEkBMF_MFcgkVunGNTAtpg_204lrXLkQ',
 'VrqWPnTPTMOIFWnC1FgF6O3NW5MzerA8wJ02272pjRfk2ARfXpPdOYYWEw',
 'PyyUEObPIwU2T1_TwLDKT9Vtu4x9twT_fVOR6M5YlrqUmSo',
 'xybM8pDZzCLcyNs7FQWt98ps9XWRIPUv_OaNBNoYaBYpnbE',
 'caLPIQ62hBU_sbRmDVDxNmwqYAD7fWYULkBTYiW5m2ZmZog',
 'tDik4owkqtRwo3SyDwDmi90AyYyrdJCf9REs6sqX-HYKVsU',
 'aZ_Qq3BtuXZAXAtYuvICveZvJOvz9xwJGyOdCUC_iDR_d1Q',
 '7_vjVmMpaVm6miPeA_wqum-7xF6Qpkk-6wLaLp9xt8u5bw',
 'W1tTkmJks74UljiwtesLNgEnQxF3nrqcNH2iYTa1c-zWEwM',
 'w2FHQJJ5Lo4pAMhnKPqS5VttCBqhuToneuH_qUEC2nV7qsvj',
 'm9SZwIAO

In [29]:
summoner_df = pd.read_csv(f'../data/{tier}_match_by_summoner.csv')
summoner_df

,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,1560,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,13,BLUE,MID,3363,4630,6657,...,0,9,1,1,4,3,12,8334,2751,LOSE
1,CNrwP4AK2Xe1SGRd-otNS5nV6TMvh7jgs2tXkSD5CEs=,1320,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,BLUE,MID,3363,3100,3115,...,1,3,0,2,0,2,0,8892,2896,LOSE
2,CNrwP4AK2Xdi2_kFJSvTtn2dBruz3jlZt1W92YA3Sjg=,1418,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,3113,3115,...,0,6,0,2,5,0,0,7683,57,LOSE
3,CNrwP4AK2Xc7lbRKnMKmbGRWJa5HoFEi1ukJgLKusn4=,1558,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,1058,2420,...,2,9,1,6,1,6,0,10677,1816,WIN
4,CNrwP4AK2XdtwndtdTggt5-DDndWH9WOTCTcDueS91o=,1740,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,3,BLUE,MID,3340,3152,2055,...,0,13,0,2,6,11,0,8925,2265,LOSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824671,CNrwP4AK2XcuLAUwGjRcPsYz7LHzsHWQx9rbXkhM8-o=,1375,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,1055,3091,...,1,7,3,8,4,3,13,13239,11841,WIN
824672,CNrwP4AK2XcJUumytI7EkP5v8HhShIWuoki3rnQOibQ=,2419,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,777,RED,MID,3364,3031,6665,...,6,7,1,15,9,5,38,20759,21222,WIN
824673,CNrwP4AK2XdMX2UtgEPveuUii4UNF1jpcsl8izOQo2U=,1001,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,134,RED,MID,3363,1056,3113,...,2,7,0,1,2,2,0,6358,401,WIN
824674,CNrwP4AK2XdJ2fmOdyFbYc5dz4Ae6E7FoDzlksEGre4=,1681,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,6610,3123,...,2,9,1,7,5,1,4,14105,13533,LOSE


In [30]:
summoner_df = summoner_df.drop_duplicates(['match_id', 'summoner_id'])
summoner_df

,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,1560,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,13,BLUE,MID,3363,4630,6657,...,0,9,1,1,4,3,12,8334,2751,LOSE
1,CNrwP4AK2Xe1SGRd-otNS5nV6TMvh7jgs2tXkSD5CEs=,1320,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,BLUE,MID,3363,3100,3115,...,1,3,0,2,0,2,0,8892,2896,LOSE
2,CNrwP4AK2Xdi2_kFJSvTtn2dBruz3jlZt1W92YA3Sjg=,1418,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,3113,3115,...,0,6,0,2,5,0,0,7683,57,LOSE
3,CNrwP4AK2Xc7lbRKnMKmbGRWJa5HoFEi1ukJgLKusn4=,1558,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,1058,2420,...,2,9,1,6,1,6,0,10677,1816,WIN
4,CNrwP4AK2XdtwndtdTggt5-DDndWH9WOTCTcDueS91o=,1740,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,3,BLUE,MID,3340,3152,2055,...,0,13,0,2,6,11,0,8925,2265,LOSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824671,CNrwP4AK2XcuLAUwGjRcPsYz7LHzsHWQx9rbXkhM8-o=,1375,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,1055,3091,...,1,7,3,8,4,3,13,13239,11841,WIN
824672,CNrwP4AK2XcJUumytI7EkP5v8HhShIWuoki3rnQOibQ=,2419,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,777,RED,MID,3364,3031,6665,...,6,7,1,15,9,5,38,20759,21222,WIN
824673,CNrwP4AK2XdMX2UtgEPveuUii4UNF1jpcsl8izOQo2U=,1001,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,134,RED,MID,3363,1056,3113,...,2,7,0,1,2,2,0,6358,401,WIN
824674,CNrwP4AK2XdJ2fmOdyFbYc5dz4Ae6E7FoDzlksEGre4=,1681,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,6610,3123,...,2,9,1,7,5,1,4,14105,13533,LOSE


In [31]:
summoner_df = summoner_df[summoner_df['result'].isin(['WIN', 'LOSE'])]
summoner_df

,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,1560,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,13,BLUE,MID,3363,4630,6657,...,0,9,1,1,4,3,12,8334,2751,LOSE
1,CNrwP4AK2Xe1SGRd-otNS5nV6TMvh7jgs2tXkSD5CEs=,1320,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,BLUE,MID,3363,3100,3115,...,1,3,0,2,0,2,0,8892,2896,LOSE
2,CNrwP4AK2Xdi2_kFJSvTtn2dBruz3jlZt1W92YA3Sjg=,1418,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,3113,3115,...,0,6,0,2,5,0,0,7683,57,LOSE
3,CNrwP4AK2Xc7lbRKnMKmbGRWJa5HoFEi1ukJgLKusn4=,1558,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,1058,2420,...,2,9,1,6,1,6,0,10677,1816,WIN
4,CNrwP4AK2XdtwndtdTggt5-DDndWH9WOTCTcDueS91o=,1740,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,3,BLUE,MID,3340,3152,2055,...,0,13,0,2,6,11,0,8925,2265,LOSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824671,CNrwP4AK2XcuLAUwGjRcPsYz7LHzsHWQx9rbXkhM8-o=,1375,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,1055,3091,...,1,7,3,8,4,3,13,13239,11841,WIN
824672,CNrwP4AK2XcJUumytI7EkP5v8HhShIWuoki3rnQOibQ=,2419,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,777,RED,MID,3364,3031,6665,...,6,7,1,15,9,5,38,20759,21222,WIN
824673,CNrwP4AK2XdMX2UtgEPveuUii4UNF1jpcsl8izOQo2U=,1001,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,134,RED,MID,3363,1056,3113,...,2,7,0,1,2,2,0,6358,401,WIN
824674,CNrwP4AK2XdJ2fmOdyFbYc5dz4Ae6E7FoDzlksEGre4=,1681,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,6610,3123,...,2,9,1,7,5,1,4,14105,13533,LOSE


In [32]:
summoner_df.columns

Index(['match_id', 'game_length_second', 'summoner_id', 'summoner_level',
       'champion_id', 'team_key', 'position', 'trinket_item', 'item_0',
       'item_1', 'item_2', 'item_3', 'item_4', 'rune_0', 'rune_1', 'spell_0',
       'spell_1', 'champion_level', 'damage_self_mitigated',
       'damage_dealt_to_objectives', 'damage_dealt_to_turrets',
       'total_damage_taken', 'total_damage_dealt',
       'total_damage_dealt_to_champions', 'time_ccing_others', 'vision_score',
       'vision_wards_bought_in_game', 'sight_wards_bought_in_game',
       'ward_kill', 'ward_place', 'turret_kill', 'kill', 'death', 'assist',
       'neutral_minion_kill', 'gold_earned', 'total_heal', 'result'],
      dtype='object')

In [33]:
cate_col = ['match_id', 'summoner_id', 'champion_id', 'team_key', 'position', 'trinket_item', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'rune_0', 'rune_1', 'spell_0', 'spell_1', 'result']
cont_col = ['game_length_second', 'summoner_level', 'champion_level', 'damage_self_mitigated', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets', 'total_damage_taken', 'total_damage_dealt', 
            'total_damage_dealt_to_champions', 'time_ccing_others', 'vision_score', 'vision_wards_bought_in_game', 'sight_wards_bought_in_game', 'ward_kill', 'ward_place', 'turret_kill', 'kill', 
            'death', 'assist', 'neutral_minion_kill', 'gold_earned', 'total_heal']

In [34]:
for col in cate_col:
    summoner_df[col] = summoner_df[col].apply(lambda x: col + '.' + str(x))
summoner_df

/tmp/ipykernel_26409/1842175635.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summoner_df[col] = summoner_df[col].apply(lambda x: col + '.' + str(x))


,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,1560,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.13,team_key.BLUE,position.MID,trinket_item.3363,item_0.4630,item_1.6657,...,0,9,1,1,4,3,12,8334,2751,result.LOSE
1,match_id.CNrwP4AK2Xe1SGRd-otNS5nV6TMvh7jgs2tXk...,1320,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.268,team_key.BLUE,position.MID,trinket_item.3363,item_0.3100,item_1.3115,...,1,3,0,2,0,2,0,8892,2896,result.LOSE
2,match_id.CNrwP4AK2Xdi2_kFJSvTtn2dBruz3jlZt1W92...,1418,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.268,team_key.RED,position.MID,trinket_item.3363,item_0.3113,item_1.3115,...,0,6,0,2,5,0,0,7683,57,result.LOSE
3,match_id.CNrwP4AK2Xc7lbRKnMKmbGRWJa5HoFEi1ukJg...,1558,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.268,team_key.RED,position.MID,trinket_item.3363,item_0.1058,item_1.2420,...,2,9,1,6,1,6,0,10677,1816,result.WIN
4,match_id.CNrwP4AK2XdtwndtdTggt5-DDndWH9WOTCTcD...,1740,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.3,team_key.BLUE,position.MID,trinket_item.3340,item_0.3152,item_1.2055,...,0,13,0,2,6,11,0,8925,2265,result.LOSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824671,match_id.CNrwP4AK2XcuLAUwGjRcPsYz7LHzsHWQx9rbX...,1375,summoner_id.ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203O...,83,champion_id.39,team_key.BLUE,position.TOP,trinket_item.3364,item_0.1055,item_1.3091,...,1,7,3,8,4,3,13,13239,11841,result.WIN
824672,match_id.CNrwP4AK2XcJUumytI7EkP5v8HhShIWuoki3r...,2419,summoner_id.ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203O...,83,champion_id.777,team_key.RED,position.MID,trinket_item.3364,item_0.3031,item_1.6665,...,6,7,1,15,9,5,38,20759,21222,result.WIN
824673,match_id.CNrwP4AK2XdMX2UtgEPveuUii4UNF1jpcsl8i...,1001,summoner_id.ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203O...,83,champion_id.134,team_key.RED,position.MID,trinket_item.3363,item_0.1056,item_1.3113,...,2,7,0,1,2,2,0,6358,401,result.WIN
824674,match_id.CNrwP4AK2XdJ2fmOdyFbYc5dz4Ae6E7FoDzlk...,1681,summoner_id.ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203O...,83,champion_id.39,team_key.BLUE,position.TOP,trinket_item.3364,item_0.6610,item_1.3123,...,2,9,1,7,5,1,4,14105,13533,result.LOSE


In [35]:
unique = np.array([])
for col in cate_col:
    unique = np.concatenate((unique, summoner_df[col].unique()))
    
unique

array(['match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=',
       'match_id.CNrwP4AK2Xe1SGRd-otNS5nV6TMvh7jgs2tXkSD5CEs=',
       'match_id.CNrwP4AK2Xdi2_kFJSvTtn2dBruz3jlZt1W92YA3Sjg=', ...,
       'spell_1.3', 'result.LOSE', 'result.WIN'], dtype=object)

In [36]:
to_index = {}
### mask index 0을 생성.
for i, u in enumerate(unique, start=1):
    to_index[u] = i

with open('../data/to_index.pkl', 'wb') as file:
    pickle.dump(to_index, file)

In [37]:
for col in cate_col:
    summoner_df[col] = summoner_df[col].map(to_index)

summoner_df

/tmp/ipykernel_26409/219301454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summoner_df[col] = summoner_df[col].map(to_index)
/tmp/ipykernel_26409/219301454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summoner_df[col] = summoner_df[col].map(to_index)
/tmp/ipykernel_26409/219301454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,1,1560,477959,266,560532,560699,560701,560707,560713,560981,...,0,9,1,1,4,3,12,8334,2751,562068
1,2,1320,477959,266,560533,560699,560701,560707,560714,560982,...,1,3,0,2,0,2,0,8892,2896,562068
2,3,1418,477959,266,560533,560700,560701,560707,560715,560982,...,0,6,0,2,5,0,0,7683,57,562068
3,4,1558,477959,266,560533,560700,560701,560707,560716,560983,...,2,9,1,6,1,6,0,10677,1816,562069
4,5,1740,477959,266,560534,560699,560701,560708,560717,560984,...,0,13,0,2,6,11,0,8925,2265,562068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824671,461376,1375,560531,83,560545,560699,560703,560709,560725,561007,...,1,7,3,8,4,3,13,13239,11841,562069
824672,119956,2419,560531,83,560557,560700,560701,560709,560738,561061,...,6,7,1,15,9,5,38,20759,21222,562069
824673,402337,1001,560531,83,560649,560700,560701,560707,560720,561089,...,2,7,0,1,2,2,0,6358,401,562069
824674,41765,1681,560531,83,560545,560699,560703,560709,560792,561082,...,2,9,1,7,5,1,4,14105,13533,562068


In [38]:
summoner_df = summoner_df.astype(int)
summoner_df.to_csv(f'../data/{tier}_match_by_summoner_mod.csv', mode='w', index=False)

# Match

In [39]:
match_df = pd.read_csv(f'../data/{tier}_match_by_match.csv')
match_df

,match_id,team_key,summoner_id,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r-fHgcIGknzMw,LOSE
1,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,xe4NOfPaQy8aUvWRPjo-9J_daUe1poo-yEWgFn6aCj2vRA,LOSE
2,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,LOSE
3,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmpP0lXG2MJ7G1pQ,WIN
4,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,vgSDdCghIp2Yk0HLfDNrL8JzET6zfihaclpOLHrtDiw7yqU,WIN
...,...,...,...,...
8246755,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,HGGccfhZJmTDsGjcJlwkUhpjdowQ2pKm74cUNtmmO52sgCE,LOSE
8246756,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,52cRsBkYTek54NCxFRe5RiotLYsAnNGbJrGQLJ3VNv146hA,WIN
8246757,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3AW3thpbDE6HcX...,LOSE
8246758,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,LOSE


In [40]:
match_df = match_df.drop_duplicates(['match_id', 'summoner_id'])
match_df

,match_id,team_key,summoner_id,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r-fHgcIGknzMw,LOSE
1,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,xe4NOfPaQy8aUvWRPjo-9J_daUe1poo-yEWgFn6aCj2vRA,LOSE
2,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,LOSE
3,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmpP0lXG2MJ7G1pQ,WIN
4,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,vgSDdCghIp2Yk0HLfDNrL8JzET6zfihaclpOLHrtDiw7yqU,WIN
...,...,...,...,...
8246755,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,HGGccfhZJmTDsGjcJlwkUhpjdowQ2pKm74cUNtmmO52sgCE,LOSE
8246756,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,52cRsBkYTek54NCxFRe5RiotLYsAnNGbJrGQLJ3VNv146hA,WIN
8246757,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3AW3thpbDE6HcX...,LOSE
8246758,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,LOSE


In [41]:
target_bool = match_df['summoner_id'].isin(target)
match_df = match_df[target_bool].reset_index(drop=True)
match_df

,match_id,team_key,summoner_id,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r-fHgcIGknzMw,LOSE
1,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,LOSE
2,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmpP0lXG2MJ7G1pQ,WIN
3,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,vgSDdCghIp2Yk0HLfDNrL8JzET6zfihaclpOLHrtDiw7yqU,WIN
4,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,O-iLgR6I7C58cMpFzyduQlJxZevdfoknb2-4bk1fp4I9dNc,WIN
...,...,...,...,...
3068073,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,MjiYweessMwizZmID6fl7HtJ2l52-kjaCHKw3BpJD4YXqw,WIN
3068074,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,KMxsTXOsx3b0g0ZndKZF3ERjnn4BlV5O56LovLidW4HIZaI,WIN
3068075,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,oPdQVPS1xVpz5WGTQqH3go_qyMoSBGNRyOkEDwPsL6qtZnE,LOSE
3068076,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3AW3thpbDE6HcX...,LOSE


In [42]:
match_df = match_df[match_df['result'].isin(['WIN', 'LOSE'])]
match_df

,match_id,team_key,summoner_id,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r-fHgcIGknzMw,LOSE
1,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,LOSE
2,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmpP0lXG2MJ7G1pQ,WIN
3,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,vgSDdCghIp2Yk0HLfDNrL8JzET6zfihaclpOLHrtDiw7yqU,WIN
4,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,O-iLgR6I7C58cMpFzyduQlJxZevdfoknb2-4bk1fp4I9dNc,WIN
...,...,...,...,...
3068073,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,MjiYweessMwizZmID6fl7HtJ2l52-kjaCHKw3BpJD4YXqw,WIN
3068074,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,KMxsTXOsx3b0g0ZndKZF3ERjnn4BlV5O56LovLidW4HIZaI,WIN
3068075,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,oPdQVPS1xVpz5WGTQqH3go_qyMoSBGNRyOkEDwPsL6qtZnE,LOSE
3068076,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3AW3thpbDE6HcX...,LOSE


In [43]:
for col in ['match_id', 'team_key', 'summoner_id']:
    match_df[col] = match_df[col].apply(lambda x: col + '.' + str(x))

match_df

/tmp/ipykernel_26409/2471060713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df[col] = match_df[col].apply(lambda x: col + '.' + str(x))


,match_id,team_key,summoner_id,result
0,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.BLUE,summoner_id.PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r...,LOSE
1,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.BLUE,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,LOSE
2,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.RED,summoner_id.WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmp...,WIN
3,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.RED,summoner_id.vgSDdCghIp2Yk0HLfDNrL8JzET6zfihacl...,WIN
4,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.RED,summoner_id.O-iLgR6I7C58cMpFzyduQlJxZevdfoknb2...,WIN
...,...,...,...,...
3068073,match_id.CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnN...,team_key.BLUE,summoner_id.MjiYweessMwizZmID6fl7HtJ2l52-kjaCH...,WIN
3068074,match_id.CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnN...,team_key.BLUE,summoner_id.KMxsTXOsx3b0g0ZndKZF3ERjnn4BlV5O56...,WIN
3068075,match_id.CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnN...,team_key.RED,summoner_id.oPdQVPS1xVpz5WGTQqH3go_qyMoSBGNRyO...,LOSE
3068076,match_id.CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnN...,team_key.RED,summoner_id.hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3A...,LOSE


In [44]:
with open('../data/to_index.pkl', 'rb') as file:
    to_index = pickle.load(file)

to_index['LOSE'] = 0
to_index['WIN'] = 1

for col in match_df.columns:
    match_df[col] = match_df[col].map(to_index)

match_df

/tmp/ipykernel_26409/426253025.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df[col] = match_df[col].map(to_index)
/tmp/ipykernel_26409/426253025.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df[col] = match_df[col].map(to_index)
/tmp/ipykernel_26409/426253025.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,match_id,team_key,summoner_id,result
0,1,560699,518555.0,0
1,1,560699,477959.0,0
2,1,560700,509625.0,1
3,1,560700,505123.0,1
4,1,560700,494939.0,1
...,...,...,...,...
3068073,477958,560699,486581.0,1
3068074,477958,560699,531954.0,1
3068075,477958,560700,538028.0,0
3068076,477958,560700,526480.0,0


In [45]:
nna = match_df['summoner_id'].notna()
match_df = match_df[nna]
match_df

,match_id,team_key,summoner_id,result
0,1,560699,518555.0,0
1,1,560699,477959.0,0
2,1,560700,509625.0,1
3,1,560700,505123.0,1
4,1,560700,494939.0,1
...,...,...,...,...
3068073,477958,560699,486581.0,1
3068074,477958,560699,531954.0,1
3068075,477958,560700,538028.0,0
3068076,477958,560700,526480.0,0


In [46]:
temp = match_df.groupby(['match_id', 'team_key']).size()
valid_indices = temp[temp > 1].index
match_df = match_df[match_df.set_index(['match_id', 'team_key']).index.isin(valid_indices)]
match_df

,match_id,team_key,summoner_id,result
0,1,560699,518555.0,0
1,1,560699,477959.0,0
2,1,560700,509625.0,1
3,1,560700,505123.0,1
4,1,560700,494939.0,1
...,...,...,...,...
3068073,477958,560699,486581.0,1
3068074,477958,560699,531954.0,1
3068075,477958,560700,538028.0,0
3068076,477958,560700,526480.0,0


In [47]:
match_df = match_df.astype(int)
match_df.to_csv(f'../data/{tier}_match_by_match_mod.csv', mode='w', index=False)

In [48]:
df = pd.read_csv(f'../data/{tier}_match_by_match_mod.csv')


In [65]:
df['match_id'].loc[1]

1

In [56]:
t = (i for i in range(10))
print(next(t))
for i in t:
    print(i)

0
1
2
3
4
5
6
7
8
9


# DATASET

In [13]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn

import itertools
import math

In [24]:
class MyDataset(Dataset):
    def __init__(self, cfg, summoner_df, match_df):
        self.cate_col = cfg.cate_col
        self.cont_col = cfg.cont_col
        self.max_seq_len = cfg.max_seq_len    ### 10
        
        self.summoner_df, self.summoner_idx = self.prepare_summoner(summoner_df)
        self.match_df, self.len, self.combinations, self.match_idx = self.prepare_match(match_df)

        self.embedding_max_idx = self.summoner_df[self.cate_col].max().max() + 1


    def prepare_summoner(self, df):
        df = df.sort_values(by='summoner_id')
        summoner_idx = []

        start = 0
        for end in self.summoner_df['summoner_id'].value_counts().sort_index().values:
            summoner_idx.append((start, start+end))
            start += end

        return df, summoner_idx
    

    def prepare_match(self, df, k=2):
        df = df.sort_values(by='match_id')
        temp = df.groupby(['match_id', 'team_key']).size()

        len = temp.__len__()
        
        combinations = [[list(element) for element in itertools.combinations(range(count), 2)] for count in df.values]

        match_idx = []
        start = 0
        for end in self.match_df['summoner_id'].value_counts().sort_index().values:
            match_idx.append((start, start+end))
            start += end
        
        return df, len, combinations, match_idx

    '''
    idx를 받으면 먼저 match 데이터에서 해당하는 match_id와 team_key에 맞는 데이터를 가져온다.
    해당하는 데이터는 최대 5개 최소 2개의 summoner_id와 1개의 result를 가지고 있다.
    가져온 summoner_id에서 2개를 뽑는 경우의 수를 구하고, 각 경우의 수에 맞게 summoner 데이터에서 해당하는 데이터를 가져온다.
    가져온 데이터는 cate, cont로 나누어져 있으며, cate는 embedding을 위해 int로, cont는 float로 변환한다.
    변환이 끝난 데이터는 A, B로 나누어 출력한다.
    '''
    def __getitem__(self, idx):
        summoner_start, summoner_end = self.summoner_idx[idx]
        summoner_df = self.summoner_df.iloc[summoner_start, summoner_end]

        match_start, match_end = self.match_idx[idx]
        match_df = self.match_df.iloc[match_start, match_end]

        seq_len = summoner_end - summoner_start
        combination = self.combinations[idx]

        axis0 = len(combination)
        axis1 = self.max_seq_len

        A['cate'] = torch.zeros(axis0, axis1, len(self.cate_col), dtype=torch.int)
        A['cont'] = torch.zeros(axis0, axis1, len(self.cate_col), dtype=torch.float)
        B['cate'] = torch.zeros(axis0, axis1, len(self.cate_col), dtype=torch.int)
        B['cont'] = torch.zeros(axis0, axis1, len(self.cate_col), dtype=torch.float)
        for i, (a, b) in enumerate(combination):
            summoner_id_a = match_df.iloc[a]['summoner_id']
            for j in :
                pass
            summoner_id_b = match_df.iloc[b]['summoner_id']
                A['cate'][i][](torch.tensor(summoner_df.loc(summoner_id_a)[self.cate_col].values, dtype=torch.int))
                A['cont'][i][](torch.tensor(summoner_df.loc(summoner_id_a)[self.cont_col].values, dtype=torch.float))
                B['cate'][i][](torch.tensor(summoner_df.loc(summoner_id_b)[self.cate_col].values, dtype=torch.int))
                B['cont'][i][](torch.tensor(summoner_df.loc(summoner_id_b)[self.cate_col].values, dtype=torch.float))


        ### A, B의 cate, cont를 10개의 seqence 길이로 초기화
        a, b = next(combination)
        A['cate'][seq_len] = torch.tensor(summoner_df.iloc(a)[self.cate_col].values)
        A['cont'][seq_len] = torch.tensor(summoner_df.iloc(a)[self.cont_col].values)
        B['cate'][seq_len] = torch.tensor(summoner_df.iloc(b)[self.cate_col].values)
        B['cont'][seq_len] = torch.tensor(summoner_df.iloc(b)[self.cont_col].values)

        for a, b in combination:
            A['cate'][seq_len] = torch.concat([A['cate'][seq_len], torch.tensor(summoner_df.iloc(a)[self.cate_col].values)], dim=0)
            A['cont'][seq_len] = torch.concat([A['cont'][seq_len], torch.tensor(summoner_df.iloc(a)[self.cont_col].values)], dim=0)
            B['cate'][seq_len] = torch.concat([B['cate'][seq_len], torch.tensor(summoner_df.iloc(b)[self.cate_col].values)], dim=0)
            B['cont'][seq_len] = torch.concat([B['cont'][seq_len], torch.tensor(summoner_df.iloc(b)[self.cont_col].values)], dim=0)

        result = torch.tensor(df['result'][0].values, dtype=torch.bool).expand(self.max_seq_len)

        return A, B, result
        

    def __len__(self):
        return 

In [26]:
class SimilarityModel(nn.Module):
    def __init__(self, config):
        super(SimilarityModel, self).__init__()
        self.embedding = nn.Embedding(config['n_layers'], config['emb_size'])

        # categorical
        self.cate_emb = nn.Embedding(cfg.total_cate_size, cfg.emb_size, padding_idx=0)
        self.cate_proj = nn.Sequential(
            nn.Linear(cfg.emb_size * cfg.cate_col_size, cate_col_hidden_size),
            nn.LayerNorm(cate_col_hidden_size),
        )

        # continuous
        self.cont_bn = nn.BatchNorm1d(cfg.cont_col_size)
        self.cont_emb = nn.Sequential(
            nn.Linear(cfg.cont_col_size, cont_col_hidden_size),
            nn.LayerNorm(cont_col_hidden_size),
        )

        # combination
        self.comb_proj = nn.Sequential(
            nn.ReLU(),
            nn.Linear(cfg.hidden_size*2, cfg.hidden_size),
            nn.LayerNorm(cfg.hidden_size)
        )


    def forward(self, input):
        input = input.transpose_(0, 1)
        embedded = self.embedding(input)
        A = embedded[0]
        B = embedded[1]
        output = F.cosine_similarity(A, B, dim=1)     

        return output

In [31]:
df = pd.read_csv(f'../data/{tier}_matches.csv')

config = {}
config['n_layers'] = int(df.drop(columns='result').max().max() + 1)
config['emb_size'] = 3

dataset = MyDataset(df)
loader = DataLoader(dataset, batch_size=5, shuffle=True)

model = SimilarityModel(config).to('cuda')

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

model.train()
for x, y in tqdm(loader):
    optimizer.zero_grad()
    output = model(x)

    # print(output, y)
    loss = loss_fun(output, y)
    loss.backward()
    optimizer.step()

  0%|          | 0/2521284 [00:00<?, ?it/s]/tmp/ipykernel_34020/378348694.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = torch.tensor(x, dtype=torch.int32, device='cuda')
100%|██████████| 2521284/2521284 [2:06:43<00:00, 331.59it/s]  


In [33]:
import joblib

joblib.dump(model, 'cossim.joblib')

['cossim.joblib']